### Import Libraries

In [3]:
import pandas as pd
import numpy as np

#### Import dataset

In [7]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

house_data = pd.read_csv(filepath_or_buffer= './data/kc_house_data.csv', dtype = dtype_dict)  
train_data = pd.read_csv('./data/kc_house_train_data.csv', dtype = dtype_dict)  
test_data = pd.read_csv('./data/kc_house_test_data.csv', dtype = dtype_dict)  

In [9]:
house_data.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  object 
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  float64
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  float64
 6   sqft_lot       21613 non-null  int32  
 7   floors         21613 non-null  object 
 8   waterfront     21613 non-null  int32  
 9   view           21613 non-null  int32  
 10  condition      21613 non-null  int32  
 11  grade          21613 non-null  int32  
 12  sqft_above     21613 non-null  int32  
 13  sqft_basement  21613 non-null  int32  
 14  yr_built       21613 non-null  int32  
 15  yr_renovated   21613 non-null  int32  
 16  zipcode        21613 non-null  object 
 17  lat            21613 non-null  float64
 18  long  

In [10]:
train_data.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17384 entries, 0 to 17383
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             17384 non-null  object 
 1   date           17384 non-null  object 
 2   price          17384 non-null  float64
 3   bedrooms       17384 non-null  float64
 4   bathrooms      17384 non-null  float64
 5   sqft_living    17384 non-null  float64
 6   sqft_lot       17384 non-null  int32  
 7   floors         17384 non-null  object 
 8   waterfront     17384 non-null  int32  
 9   view           17384 non-null  int32  
 10  condition      17384 non-null  int32  
 11  grade          17384 non-null  int32  
 12  sqft_above     17384 non-null  int32  
 13  sqft_basement  17384 non-null  int32  
 14  yr_built       17384 non-null  int32  
 15  yr_renovated   17384 non-null  int32  
 16  zipcode        17384 non-null  object 
 17  lat            17384 non-null  float64
 18  long  

In [11]:
test_data.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4229 entries, 0 to 4228
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4229 non-null   object 
 1   date           4229 non-null   object 
 2   price          4229 non-null   float64
 3   bedrooms       4229 non-null   float64
 4   bathrooms      4229 non-null   float64
 5   sqft_living    4229 non-null   float64
 6   sqft_lot       4229 non-null   int32  
 7   floors         4229 non-null   object 
 8   waterfront     4229 non-null   int32  
 9   view           4229 non-null   int32  
 10  condition      4229 non-null   int32  
 11  grade          4229 non-null   int32  
 12  sqft_above     4229 non-null   int32  
 13  sqft_basement  4229 non-null   int32  
 14  yr_built       4229 non-null   int32  
 15  yr_renovated   4229 non-null   int32  
 16  zipcode        4229 non-null   object 
 17  lat            4229 non-null   float64
 18  long    